In [90]:
# two examples (euro, train) are based on
# code from Think Bayes 2 by Allen B. Downey
# original source: https://allendowney.github.io/ThinkBayes2/
# license: https://creativecommons.org/licenses/by-nc-sa/4.0/

from scipy.stats import binom
from scipy.special import binom as binom_coef
from empiricaldist import Pmf
import warnings
import altair as alt
import numpy as np
import pandas as pd

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

n = 2
p = 0.5
k = 1

ks = np.arange(n+1)

binom.pmf(k, n, p)
binom.pmf(ks, n, p)

array([0.25, 0.5 , 0.25])

In [60]:
def make_binomial(n, p):
    ks = np.arange(n+1)
    ps = binom.pmf(ks, n, p)
    return Pmf(ps, ks)

In [61]:
make_binomial(2, 0.5)

,probs
0,0.25
1,0.50
2,0.25


In [62]:
hypos = np.linspace(0, 1, 101)
prior = Pmf(1, hypos)

likelihood_heads = hypos
likelihood_tails = 1 - hypos

likelihood = {
    "H": likelihood_heads,
    "T": likelihood_tails
}

dataset = 'H' * 140 + 'T' * 110

In [65]:
def update_euro(pmf, dataset, dists):
    pmf = pmf.copy()
    for i, data in enumerate(dataset):
        pmf *= likelihood[data]
        pmf.normalize()
        dist = pd.DataFrame(pmf.copy())
        dist["run"] = i
        dists.append(dist)

dists = []
update_euro(prior, dataset, dists)

In [66]:
log = pd.concat(dists)
log.columns = ["p", "run"]
log = log.reset_index()

alt.data_transformers.disable_max_rows()
alt.Chart(log[log.run % 5 == 1]).encode(x="index", y="p", color="run").mark_line(opacity=0.2)

alt.Chart(...)

In [82]:
def likelihood(data, n):
    return np.append(
        np.repeat(0, data),
        1/np.arange(data, n)
    )
    
def update_train(pmf, dataset, dists, n):
    pmf = pmf.copy()
    pmf.normalize()
    for i, data in enumerate(dataset):
        pmf *= likelihood(data, n)
        pmf.normalize()
        dist = pd.DataFrame(pmf.copy())
        dist["run"] = i
        dists.append(dist)

n = 300
dists = []
pmf = Pmf(1, np.arange(n))
update_train(pmf, [60, 90, 70, 100], dists, n)

log = pd.concat(dists)
log.columns = ["p", "run"]
log = log.reset_index()

alt.Chart(log).encode(x="index", y="p", color="run").mark_line(opacity=1)

alt.Chart(...)

In [103]:
def likelihood(identified, hypo, n):
    if identified > marked or marked > recaptured or recaptured > n:
        raise Exception
    a = binom_coef(marked, identified)
    b = binom_coef(hypo - marked, recaptured - identified)
    c = binom_coef(hypo, recaptured)
    return a*b/c
    
def update_wolves(pmf, dataset, dists, n):
    pmf = pmf.copy()
    pmf.normalize()
    for i, data in enumerate(dataset):
        pmf *= likelihood(data, pmf.index.values, n)
        pmf.normalize()
        dist = pd.DataFrame(pmf.copy())
        dist["run"] = i
        dists.append(dist)

n = 100
marked = 10
recaptured = 20        

dists = []
pmf = Pmf(1, np.arange(n))
update_wolves(pmf, [5, 4, 3, 2], dists, n)

log = pd.concat(dists)
log.columns = ["p", "run"]
log = log.reset_index()

alt.Chart(log).encode(x="index", y="p", color="run").mark_line(opacity=1)

alt.Chart(...)